In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import nept


In [ ]:
from run import info

In [ ]:
infos = [info.r066d2, info.r066d3, info.r066d4, info.r066d5, info.r066d6, info.r066d7, info.r066d8]

In [ ]:
def combine_behavior(infos):
    durations = dict(u=[], shortcut=[], novel=[])
    n_sessions = 0

    trials = []
    togethers = []

    for info in infos:
        print(info.session_id)

        events, position, spikes, lfp, lfp_theta = get_data(info)

        t_start = info.task_times['phase3'].start
        t_stop = info.task_times['phase3'].stop

        sliced_pos = position.time_slice(t_start, t_stop)

        feeder1_times = []
        for feeder1 in events['feeder1']:
            if t_start < feeder1 < t_stop:
                feeder1_times.append(feeder1)

        feeder2_times = []
        for feeder2 in events['feeder2']:
            if t_start < feeder2 < t_stop:
                feeder2_times.append(feeder2)

        path_pos = get_zones(info, sliced_pos)

        trials_idx = get_trial_idx(path_pos['u'].time, path_pos['shortcut'].time, path_pos['novel'].time,
                                   feeder1_times, feeder2_times, t_stop)

        trials.append(trials_idx)
        
        trial_epoch = nept.Epoch([trials_idx['start_trials'], trials_idx['stop_trials']])

        n_sessions += 1

        for key in durations:
            for trial in trials_idx[key]:
                durations[key].append(trial_epoch.durations[trial[0]])

        for trial in trials:
            togethers.append(sorted(trial['u'] + trial['shortcut'] + trial['novel']))

    return durations, trials, togethers, n_sessions


def bytrial_counts(togethers, min_length):
    """Finds the behavioral choice by trial for all animals combined.

    Parameters
    ----------
    togethers : list
    min_length : int

    Returns
    -------
    means : dict
        With u, shortcut, novel as keys
    err : dict
        With u, shortcut, novel as keys

    """
    bytrial = dict(u=[], shortcut=[], novel=[])
    for trial in range(min_length):
        for key in bytrial:
            bytrial[key].append([])

    for session in togethers:
        for trial in range(min_length):
            for key in bytrial:
                if session[trial][1] == key:
                    bytrial[key][trial].append(1)
                else:
                    bytrial[key][trial].append(0)

    means = dict(u=[], shortcut=[], novel=[])
    err = dict(u=[], shortcut=[], novel=[])
    for trial in range(min_length):
        for key in bytrial:
            means[key].append(np.mean(bytrial[key][trial]))
            err[key].append(np.std(bytrial[key][trial]))

    return means, err

In [ ]:
from loading_data import get_data
from utils_maze import get_trial_idx, get_zones
import pandas as pd
import os
from utils_plotting import plot_proportions, plot_durations, plot_bytrial
import scipy.stats as stats

In [ ]:
output_filepath = 'e:/code/emi_shortcut/plots'

In [ ]:
proportion_filename = 'all-behavior_proportions.png'
duration_filename = 'all-behavior_durations.png'
bytrial_filename = 'all-behavior_bytrial.png'

total_n_sessions = 0

durations_together = dict(trajectory=[], value=[])
trials_together = dict(trajectory=[], value=[])

durations, trials, togethers, n_sessions = combine_behavior(infos)
total_n_sessions += n_sessions

for key in durations:
    for val in durations[key]:
        durations_together['trajectory'].append(key)
        durations_together['value'].append(val)

for key in ['u', 'shortcut', 'novel']:
    for trial in trials:
        trials_together['trajectory'].append(key)
        trials_together['value'].append(len(trial[key])/float(len(trial['start_trials'])))

df_durations = pd.DataFrame(data=durations_together)
df_trials = pd.DataFrame(data=trials_together)

# proportion_savepath = os.path.join(output_filepath, proportion_filename)
# plot_proportions(df_trials, proportion_savepath, total_n_sessions, early_late=False, figsize=(8, 5), savefig=False)

# duration_savepath = os.path.join(output_filepath, duration_filename)
# plot_durations(df_durations, duration_savepath, total_n_sessions, early_late=False, figsize=(8, 5), fliersize=3, savefig=False)

min_length = 30
means, sems = bytrial_counts(togethers, min_length)

bytrial_savepath = os.path.join(output_filepath, bytrial_filename)
plot_bytrial(means, sems, total_n_sessions, bytrial_savepath, figsize=(4, 5), savefig=False)

In [ ]:
a = np.array([1,2,3,4,5])

In [ ]:
a.mean()

In [ ]:
a.std()